# Задание 1

In [40]:
%pylab inline

import warnings
import numpy as np
import pandas as pd
from copy import deepcopy
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import cross_val_score
from sklearn.base import ClassifierMixin, BaseEstimator

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

Populating the interactive namespace from numpy and matplotlib


Считывание данных

In [2]:
with open('SMSSpamCollection.txt', 'r') as f:
    s = f.readline()
    print(s)

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...



Судя по всему, метки классов стоят в начале строки перед табуляцией.

In [3]:
def isspam(label):
    if label=='ham':
        return 0
    else:
        return 1

with open('SMSSpamCollection.txt', 'r') as f:
    labels = []
    texts = []
    for line in f:
        mas = line.split('\t')
        labels.append(isspam(mas[0]))
        texts.append('\t'.join(mas[1:]))

In [4]:
print(texts[:10])
print(labels[:10])

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\n', 'Ok lar... Joking wif u oni...\n', "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's\n", 'U dun say so early hor... U c already then say...\n', "Nah I don't think he goes to usf, he lives around here though\n", "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, \xc2\xa31.50 to rcv\n", 'Even my brother is not like to speak with me. They treat me like aids patent.\n', "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune\n", 'WINNER!! As a valued network customer you have been selected to receivea \xc2\xa3900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 h

In [6]:
vocabulary = CountVectorizer().fit(texts)
X_train = vocabulary.transform(texts)
y_train = labels

In [7]:
print(X_train[0])
print(X_train.shape)

  (0, 1082)	1
  (0, 1316)	1
  (0, 1765)	1
  (0, 1767)	1
  (0, 2061)	1
  (0, 2338)	1
  (0, 3571)	1
  (0, 3615)	1
  (0, 3655)	1
  (0, 4114)	1
  (0, 4374)	1
  (0, 4501)	1
  (0, 5571)	1
  (0, 5958)	1
  (0, 7694)	1
  (0, 8084)	1
  (0, 8324)	1
  (0, 8548)	1
(5574, 8713)


Посчитаем скор логистической регрессии при кросс-валидации на трейне

In [8]:
model = LogisticRegression()
scores = cross_val_score(model, X_train, y_train, scoring='f1', cv=10)

result = np.mean(scores)
print 'Mean score is ', result 

Mean score is  0.933348526858


Проверим на нашем маленьком тесте

In [59]:
model.fit(X_train, y_train)

with open('test.txt', 'r') as f:
    test_texts = f.readlines()
    
X_test = vocabulary.transform(test_texts)

In [11]:
print(X_test[0])

  (0, 789)	1
  (0, 1828)	1
  (0, 2067)	1
  (0, 3380)	1
  (0, 3414)	1
  (0, 3972)	1
  (0, 5400)	1
  (0, 5453)	1
  (0, 5511)	1
  (0, 5849)	1
  (0, 6504)	1
  (0, 7545)	1
  (0, 7770)	1
  (0, 7806)	2
  (0, 7986)	1
  (0, 8124)	1
  (0, 8674)	2


In [12]:
result = model.predict(X_test)
print result

[1 1 0 0 0]


Насколько я могу судить, ответы не совсем верные: как минимум 4 письмо - явный спам. Ну как-то не удалось. Общая картина верная

Напишем сразу функцию для подсчета скоров, чтобы не переписывать по 10 раз.

In [51]:
def check_range(trans, model):
    for ngram_range in [(1, 1), (2, 2), (3, 3), (1, 3)]:
        X_train = trans(ngram_range=ngram_range).fit_transform(texts)
        print ("%.3f" % np.mean(cross_val_score(model, X_train, y_train, scoring='f1', cv=10))),

Да, надо было 2 знака после запятой, но так удобнее сравнивать.

In [53]:
check_range(CountVectorizer, LogisticRegression())

0.933 0.822 0.725 0.925


In [54]:
check_range(CountVectorizer, MultinomialNB())

0.927 0.645 0.379 0.888


In [55]:
check_range(TfidfVectorizer, LogisticRegression())

0.853 0.345 0.166 0.648


Проанализируем различные методы. Скоры выводятся в порядке: униграммы, биграммы, триграммы, все вместе.

In [52]:
transes = {CountVectorizer: 'CountVectorizer', TfidfVectorizer: 'TfidfVectorizer'}
models = {LogisticRegression(): 'LogisticRegression', MultinomialNB(): 'MultinomialNB', RandomForestClassifier():
          'RandomForestClassifier', SVC(kernel='linear'): 'Linear SVM'}         

for trans in transes.keys():
    for model in models.keys():
        print transes[trans], models[model], '\t', check_range(trans, model)

CountVectorizer MultinomialNB 	0.927 0.645 0.379 0.888 None
CountVectorizer Linear SVM 	0.938 0.855 0.759 0.930 None
CountVectorizer LogisticRegression 	0.933 0.822 0.725 0.925 None
CountVectorizer RandomForestClassifier 	0.881 0.843 0.743 0.837 None
TfidfVectorizer MultinomialNB 	0.840 0.748 0.633 0.725 None
TfidfVectorizer Linear SVM 	0.938 0.820 0.734 0.916 None
TfidfVectorizer LogisticRegression 	0.853 0.345 0.166 0.648 None
TfidfVectorizer RandomForestClassifier 	0.885 0.840 0.762 0.833 None


Выводы: Итак, из рассмотренных моделей Linear SVM работает лучше всех, хотя LogisticRegression не сильно отстает при применении CountVectorizer. Linear SVM спокойно отнеслась к Tf-idf - для униграмм ее скор вообще не изменился. 

Наивный Байес отлично отработал только с CountVectorizer на униграммах, почти во всех остальных местах его скор наименьший. Также видно, что он очень плохо работает с биграммами, и еще хуже с тиграммами, что ожидаемо - ему нужно как можно больше статистики, а по биграммам и триграммам ее намного меньше, чем по униграммам. 

Логистическая регрессия при применении CountVectorizer менее чувствительна к использованию n-грамм, чем Навный Байес. Также отмечу очень сильный спад логистической регрессии на n-граммах при росте n при применении Tfidf.

Random Forest работает достаточно стабильно и вполне неплохо по сравнению с другими. Выбор векторизации на него почти не повлиял, да и колебания для различных n-грамм менее драматичны.

Вообще заметно, что на всех моделях Tf-idf дает понижение скора. Это связано с тем, что при определении спама ключевыми являются как раз такие слова, которые часто встречаются (в этом же самом спаме как минимум). Tf-idf занижает значимость этих слов, что нам совсем не нужно.

В общем, из рассмотренных моделей выбираю Linear SVM для униграмм, скор у нее стабильно выше вообще почти во всех комбинациях. Проверим, улучшилось ли что-то для нашего маленького теста

In [57]:
svm = SVC(kernel='linear').fit(X_train, y_train)
print(svm.predict(X_test))

[1 1 0 0 0]


Результат такой же не идеальный.

Выводы (продолжение): Tf-idf не самая удачная идея для выделения спама. n-граммы при n > 1 дают только ухудшение результата, что связано с недостатком статистики для моделей. Возможно, улучшение могут дать другие методы векторизации, отбор существенных признаков.